<a href="https://colab.research.google.com/github/The-Unindented/Gan_project/blob/main/DL_Gan2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from tensorflow.keras.datasets import mnist
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
x_train = (x_train.astype(np.float32) - 127.5) / 127.5

In [ ]:
x_train = x_train.reshape(-1, 784)
x_test = x_test.reshape(-1, 784)

In [ ]:
x_train = torch.tensor(x_train, dtype=torch.float32)
x_test = torch.tensor(x_test, dtype=torch.float32)

In [ ]:
# generator which takes input vector of size 64 and output an image of size 784 pixel
G = nn.Sequential(
    nn.Linear(64, 256),
    nn.ReLU(),
    nn.Linear(256, 256),
    nn.ReLU(),
    nn.Linear(256, 784),
    nn.Tanh()
).to(device)

# discriminator which takes image of 784 pixels and output the probability of being a real image
D = nn.Sequential(
    nn.Linear(784, 256),
    nn.ReLU(),
    nn.Linear(256, 256),
    nn.ReLU(),
    nn.Linear(256, 1),
    nn.Sigmoid()
).to(device)

In [ ]:
def plot_fake_images(epoch, fake_images):
    fake_images = fake_images.view(-1, 28, 28).cpu().data.numpy()
    fig, ax = plt.subplots(8, 8, figsize=(8, 8))
    for i in range(8):
        for j in range(8):
            ax[i, j].imshow(fake_images[i * 8 + j], cmap='gray')
            ax[i, j].axis('off')
    plt.suptitle(f'Fake Images at Epoch {epoch + 1}')
    plt.show()

In [ ]:
# Hyperparameters
batch_size = 128
z_dim = 64
lr = 0.0002
epochs = 100


# Optimizers for both generator and discriminator
optimizer_G = optim.Adam(G.parameters(), lr=lr)
optimizer_D = optim.Adam(D.parameters(), lr=lr)

# Loss function (Binary Cross Entropy)
criterion = nn.BCELoss()

# Training loop
for epoch in range(epochs):
    for i in range(0, len(x_train), batch_size):
        # Get real images
        real_images = x_train[i:i + batch_size].to(device)

        # Labels for real (1) and fake (0) images
        real_labels = torch.ones(len(real_images), 1).to(device)
        fake_labels = torch.zeros(len(real_images), 1).to(device)

        # Train Discriminator

        # Real images
        outputs_real = D(real_images)
        loss_real = criterion(outputs_real, real_labels)

        # Fake images
        noise = torch.randn(len(real_images), z_dim).to(device)
        fake_images = G(noise)
        outputs_fake = D(fake_images)
        loss_fake = criterion(outputs_fake, fake_labels)

        # Total discriminator loss
        loss_D = loss_real + loss_fake
        optimizer_D.zero_grad()
        loss_D.backward()
        optimizer_D.step()


        # Train Generator

        # Generate fake images and compute generator loss
        noise = torch.randn(len(real_images), z_dim).to(device)
        fake_images = G(noise)
        outputs_fake = D(fake_images)
        loss_G = criterion(outputs_fake, real_labels)

        # We want the generator to fool the discriminator
        optimizer_G.zero_grad()
        loss_G.backward()
        optimizer_G.step()



    print(f"Epoch [{epoch+1}/{epochs}], Loss D: {loss_D.item():.4f}, Loss G: {loss_G.item():.4f}")
    plot_fake_images(epoch, G(noise))